In [11]:
import arviz as az
import numpy as np
import pymc as pm

D = 50  # Cantidad de predictores
D0 = 5  # Cantidad de predictores cuyo 'beta' es distinto de 0

# Armamos el generador (para que los resultados sean reproducibles)
SEED = 12345
rng = np.random.default_rng(SEED)

INTERCEPT = rng.uniform(-3, 3)

BETAS = np.zeros(D)
BETAS[:D0] = rng.choice([-1, 1], size=D0) * rng.normal(5, 1, size=D0)

N = 100

X = rng.normal(size=(N, D))

SIGMA = 1.

mu = INTERCEPT + np.dot(X, BETAS) 
y = rng.normal(mu, SIGMA, size=N)

X.shape, mu.shape, y.shape

((100, 50), (100,), (100,))